In [1]:
try:
    import piplite
    await piplite.install(["ipywidgets==8.1.0", "ipyleaflet==0.17.3", "emfs:here_search_demo-0.9.1-py3-none-any.whl"], keep_going=True)
    api_key = "<YOUR HERE API KEY>"
except ImportError:
    api_key = None

## Browse and Derived place taxonomy

`api.browse()` supports a set of places filters around [category](https://developer.here.com/documentation/geocoding-search-api/dev_guide/topics-places/places-category-system-full.html), [cusine](https://developer.here.com/documentation/geocoding-search-api/dev_guide/topics-places/food-types-category-system-full.html), [chain](https://developer.here.com/documentation/geocoding-search-api/dev_guide/topics-places/places-chain-system-full.html) IDs.

For example, to get the nearby places with on of the category IDs `700-7600-0000` (Fueling Station), `700-7600-0116` (Petrol-Gasoline Station) or `700-7600-0444` (Hydrogen Fuel Station), on can send the following Browse request:

In [2]:
from here_search.demo.api import API, HTTPSession

api = API(api_key=api_key)
session = HTTPSession()
resp = await api.browse(latitude=52, longitude=13, limit=4, 
                        categories=["700-7600-0000", "700-7600-0116", "700-7600-0444"], session=session)
resp.req.full

'https://browse.search.hereapi.com/v1/browse?at=52%2C13&categories=700-7600-0000%2C700-7600-0116%2C700-7600-0444&limit=4&apiKey=...'

In [3]:
resp.titles

['ARAL', 'TOTAL', 'Raiffeisen', 'Shell']

The place filters multi-values allows the support for simple derived ontologies. For example:

In [4]:
from here_search.demo.entity.place import PlaceTaxonomyExample
from inspect import getsource
from IPython.display import Code

Code(getsource(PlaceTaxonomyExample))

class PlaceTaxonomyExample:
    items, icons = zip(
        *[
            #                --------------------------------------------------------------------
            #                | item name | categories     | food types | chains  | icon         |
            #                --------------------------------------------------------------------
            (PlaceTaxonomyItem("gas", ["700-7600-0000", "700-7600-0116", "700-7600-0444"], None, None), "fa-gas-pump"),
            (PlaceTaxonomyItem("eat", ["100"], None, None), "fa-utensils"),
            (PlaceTaxonomyItem("sleep", ["500-5000"], None, None), "fa-bed"),
            (PlaceTaxonomyItem("park", ["400-4300", "800-8500"], None, None), "fa-parking"),
            (PlaceTaxonomyItem("ATM", ["700-7010-0108"], None, None), "fa-euro-sign"),
            (PlaceTaxonomyItem("pizza", None, ["800-057"], None), "fa-pizza-slice"),
            (PlaceTaxonomyItem("fastfood", None, None, ["1566", "1498"]), "fa-hamburger"),
        ]
    )
    taxonomy = PlaceTaxonomy("example", items)

In [5]:
resp = await api.browse(latitude=52, longitude=13, limit=4, **PlaceTaxonomyExample.taxonomy.pizza.mapping, session=session)
display(resp.req.full)
resp.titles

'https://browse.search.hereapi.com/v1/browse?at=52%2C13&foodTypes=800-057&limit=4&apiKey=...'

['Super Pizza', 'Il-Mulino-Due', 'Wasserpfeifchen', 'First Grill']

## Request & Response objects

The `api.get()` method is called by `api.browse()` and the other endpoint methods. It takes a `Request` object and returns a `Response` object.

In [6]:
from here_search.demo.entity.request import Endpoint, Request, Response
from here_search.demo.api import base_url

req = Request(Endpoint.REVGEOCODE, base_url[Endpoint.REVGEOCODE], {"at": "52,13", "limit": 1})
await api.get(req, session)

Response(req=Request(endpoint=<Endpoint.REVGEOCODE: 6>, url='https://revgeocode.search.hereapi.com/v1/revgeocode', params={'at': '52,13', 'limit': 1, 'apiKey': '...'}, x_headers=None), data={'items': [{'title': 'Niedergörsdorfer Allee, 14913 Niedergörsdorf, Deutschland', 'id': 'here:af:streetsection:.GSuwaNujk32oWouTtr19D', 'resultType': 'street', 'address': {'label': 'Niedergörsdorfer Allee, 14913 Niedergörsdorf, Deutschland', 'countryCode': 'DEU', 'countryName': 'Deutschland', 'stateCode': 'BB', 'state': 'Brandenburg', 'countyCode': 'TF', 'county': 'Teltow-Fläming', 'city': 'Niedergörsdorf', 'street': 'Niedergörsdorfer Allee', 'postalCode': '14913'}, 'position': {'lat': 52.00028, 'lng': 13.00166}, 'distance': 118, 'mapView': {'west': 13.00058, 'south': 51.98972, 'east': 13.00786, 'north': 52.00263}}]}, x_headers={'X-Request-Id': 'REQ-67940aeb-1198-4114-b574-32db7da10df1', 'X-Correlation-ID': '0e5596dd-c381-4f46-9e8c-d5c3d2578106'})